<a href="https://colab.research.google.com/github/oxygen0605/ImageClassification/blob/master/deep_cnn_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratory環境の初期設定

## インスタンスのマシンスペック 

In [0]:
!nvidia-smi
!cat /proc/driver/nvidia/gpus/0000:00:04.0/information
!cat /etc/issue
!cat /proc/cpuinfo

Sun Aug 18 03:56:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    38W /  70W |   7769MiB / 15079MiB |     73%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Google Driveにマウント

In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Deep CNN (CIFAR-10)

## モデルの生成

In [0]:
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Dense, BatchNormalization
from keras.layers import Input
from keras.layers.core import Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras import regularizers

def deep_cnn(input_shape, num_classes):
    inputs = Input(shape = input_shape)
    
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(inputs)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = MaxPooling2D()(x)
    x = Dropout(0.25)(x)

    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = BatchNormalization()(x)
    x = Conv2D(512,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(512,(3,3),padding = "SAME",activation= "relu")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.5)(x)
    y  = Dense(num_classes, activation = "softmax")(x)

    return Model(input = inputs, output = y)

Using TensorFlow backend.


## CIFAR10 データセットの用意

In [0]:
import keras
from keras.datasets import cifar10

class CIFAR10Dataset():
	def __init__(self):
		self.image_shape = (32, 32, 3)
		self.num_classes = 10
		
	def preprocess(self, data, label_data=False):
		if label_data:
			# conver class number to one-hot vector
			data = keras.utils.to_categorical(data, self.num_classes)
		
		else:
			data = data.astype("float32")
			data /= 255 #convert the value to 0 ~ 1 scale
			shape = (data.shape[0],) + self.image_shape
			data = data.reshape(shape)
			
		return data
	
	def get_batch(self):
		# x: data, y: lebel
		(x_train, y_train), (x_test, y_test) = cifar10.load_data()
		
		x_train, x_test = [self.preprocess(d) for d in [x_train, x_test]]
		y_train, y_test = [self.preprocess(d, label_data=True) for d in
					 [y_train, y_test]]
		
		return x_train, y_train, x_test, y_test

## Training, Evaluation用クラスの定義

In [0]:
import os
from keras.models import load_model
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import RMSprop, Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

from keras.callbacks import TensorBoard

class Trainer():
	
	def __init__(self, model, loss, optimizer, logdir = './'):
		self._target = model
		self._target.compile(
				loss=loss, optimizer=optimizer, metrics=["accuracy"]
				)
		self.verbose = 1 # visualize progress bar: 0(OFF), 1(On), 2(On:each data) 
		#self.log_dir = os.path.join(os.path.dirname(__file__), logdir)
		self.log_dir = os.path.join(logdir)
		self.model_file_name = "model_file.hdf5"
	
	def train_with_data_augmentation(self, 
            x_train, y_train, batch_size, epochs, validation_split, lr_scheduler):
		"""
        fit(ImageDataGeneratorクラス)
        与えられたサンプルデータに基づいて，データに依存する統計量を計算します． 
        featurewise_center，featurewise_std_normalization，
        または，zca_whiteningが指定されたときに必要です．
        
        fit_generator(Modelクラス)
         Pythonジェネレータ（またはSequenceのインスタンス）によりバッチ毎に生成されたデータでモデルを訓練します．
         本ジェネレータは効率性のためモデルに並列して実行されます．例えば，モデルをGPUで学習させながら
         CPU上で画像のリアルタイムデータ拡張を行うことができるようになります．
        """
		"""
		# remove previous execution
		if os.path.exists(self.log_dir):
			import shutil
			shutil.rmtree(self.log_dir) 
		os.mkdir(self.log_dir)
		"""
        
		datagen = ImageDataGenerator(
			featurewise_center=False,                  # set input mean to 0 over the dataset
            samplewise_center=False,             # set each sample mean to 0
            featurewise_std_normalization=False, # divide inputs by std
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,                 # apply ZCA whitening
            rotation_range=20,                   # randomly rotate images in the range (0~180)
            width_shift_range=0.2,               # randomly shift images horizontally
            height_shift_range=0.2,              # randomly shift images vertically
            zoom_range = 0.2,
            channel_shift_range = 0.2,
            horizontal_flip=True,                # randomly flip images
            vertical_flip=False                  # randomly flip images
		)
		
    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
		datagen.fit(x_train)
		
    # for reproducibility
		np.random.seed(1671)
    #split for validation data
		indices = np.arange(x_train.shape[0])
		np.random.shuffle(indices)
		
		#シャッフルしたindicesの後ろからvalidataion_size分だけvalidationに回す
		validation_size = int(x_train.shape[0] * validation_split)
		x_train, x_valid = \
            x_train[indices[:-validation_size], :], \
            x_train[indices[-validation_size:], :]
		y_train, y_valid = \
            y_train[indices[:-validation_size], :], \
            y_train[indices[-validation_size:], :]
        
    # training        
		model_path = os.path.join(self.log_dir, self.model_file_name)
		self._target.fit_generator(
            datagen.flow(x_train, y_train, batch_size=batch_size),
            steps_per_epoch=x_train.shape[0] // batch_size,
            epochs=epochs,
            validation_data=(x_valid, y_valid),
            callbacks=[
                LearningRateScheduler(lr_scheduler),
                TensorBoard(log_dir=self.log_dir),
                ModelCheckpoint(model_path, save_best_only=True)
            ],
            verbose=self.verbose,
            workers=4
        )
		

class Evaluator():
    
    def __init__(self, result_file_path="./prediction_result.csv"):
        self.result_file_path="./prediction_result.csv"
        
    def simple_evaluate(self, model, x_test, label):
        print("start evaluation...")
        score = model.evaluate(x_test, y_test, verbose=1)
        print("Test loss:", score[0])
        print("Test accuracy:", score[1])
    
    def tta_evaluate(self, model, x_test, batch_size = 2500, tta_epochs = 2):
        print("batch size (TTA): "+str(batch_size))
        print("epochs (TTA): "+str(tta_epochs))
        tta = TTA()
        tta_pred = tta.predict(model, x_test, batch_size, epochs = tta_epochs)
        print("Test accuracy(TTA): ",end = "")
        print( accuracy_score( np.argmax(tta_pred,axis = 1) , np.argmax(y_test,axis = 1)))    

## 学習率減衰

In [0]:
def learning_rate_schedule_for_Adam(epoch):
	lr = 0.001
	if(epoch >= 100): lr = 0.0002 #100
	if(epoch >= 140): lr = 0.0001 #140
	return lr

## 実行

In [0]:
from keras.optimizers import Adam

# create dataset
dataset = CIFAR10Dataset()
x_train, y_train, x_test, y_test = dataset.get_batch()

# create model
model = deep_cnn(dataset.image_shape, dataset.num_classes)

# train the model
# RMSprpの方がいいかもしれない
trainer = Trainer(model, loss="categorical_crossentropy", optimizer=Adam())
#trainer.simple_train(x_train, y_train, batch_size=500, epochs=10, validation_split=0.2)
trainer.train_with_data_augmentation(
          x_train, y_train, batch_size=500, epochs=150, validation_split=0.2, 
          lr_scheduler=learning_rate_schedule_for_Adam)

# show result
evaluator = Evaluator()
score = evaluator.simple_evaluate(model, x_test, y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
W0817 19:05:53.672603 140310145603456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0817 19:05:58.751287 140310145603456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0817 19:05:58.753152 140310145603456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter i

Epoch 1/150
80/80 [==============================] - 55s 686ms/step - loss: 1.9567 - acc: 0.2368 - val_loss: 5.3625 - val_acc: 0.1634
Epoch 2/150
80/80 [==============================] - 37s 463ms/step - loss: 1.6568 - acc: 0.3676 - val_loss: 2.5036 - val_acc: 0.2863
Epoch 3/150
80/80 [==============================] - 37s 462ms/step - loss: 1.4858 - acc: 0.4505 - val_loss: 1.9913 - val_acc: 0.4123
Epoch 4/150
80/80 [==============================] - 38s 473ms/step - loss: 1.3447 - acc: 0.5136 - val_loss: 3.0249 - val_acc: 0.3684
Epoch 5/150
80/80 [==============================] - 37s 464ms/step - loss: 1.2415 - acc: 0.5575 - val_loss: 1.7186 - val_acc: 0.4914
Epoch 6/150
80/80 [==============================] - 36s 447ms/step - loss: 1.0870 - acc: 0.6125 - val_loss: 1.0276 - val_acc: 0.6331
Epoch 7/150
80/80 [==============================] - 37s 463ms/step - loss: 1.0317 - acc: 0.6307 - val_loss: 1.0277 - val_acc: 0.6392
Epoch 8/150
80/80 [==============================] - 37s 461ms